In [27]:
# ! pip install timm

In [28]:
import timm 
import torch
import torchsummary as ts
from pprint import pprint
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

In [29]:
avail_pretrained_models = timm.list_models(pretrained=True)
pprint(avail_pretrained_models[:10])

['adv_inception_v3',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224']


In [30]:
# search models
all_vit_models = timm.list_models('vit_*',pretrained=True)
pprint(all_vit_models[:10])

['vit_base_patch16_224',
 'vit_base_patch16_224_in21k',
 'vit_base_patch16_224_miil',
 'vit_base_patch16_224_miil_in21k',
 'vit_base_patch16_384',
 'vit_base_patch32_224',
 'vit_base_patch32_224_in21k',
 'vit_base_patch32_384',
 'vit_base_r50_s16_224_in21k',
 'vit_base_r50_s16_384']


In [31]:
model = timm.create_model('resnet18',num_classes=10,features_only=True)
model.to(device)
x     = torch.randn(1, 3, 224, 224).to(device)
feature = model(x)

print(len(feature))
print(feature[0].shape)

# ts.summary(model,input_size=(3,224,224),device="cuda")


5
torch.Size([1, 64, 112, 112])


In [32]:
model = timm.create_model('vit_base_patch16_224',num_classes=512)
model.to(device)
model.eval()
x     = torch.randn(1, 3, 224, 224).to(device)
feature = model(x)

pprint(feature)
print(len(feature))
print(feature.shape)


tensor([[-1.0405,  0.3963,  0.4848,  0.3013,  1.0827, -0.0118, -0.0156, -0.1426,
         -0.6735,  0.2107, -0.3905, -1.3386,  0.0738, -0.8147,  0.1221,  0.3698,
          0.4081, -0.1964,  0.3156,  0.4969, -0.1092,  0.3339,  0.6599, -0.6402,
          0.9963, -0.2344,  0.3304, -0.8048,  0.8812, -0.5152,  0.1694,  1.2526,
         -0.3484,  0.1725,  0.7009,  0.1702,  0.0087,  0.8480, -0.2615, -0.2875,
          1.2243,  0.0043, -0.0299, -0.0502,  0.3533, -0.5545, -1.1746,  0.4701,
         -0.0138, -0.9705, -0.3869,  0.6597, -0.5961,  0.4510, -0.3602,  0.8705,
          0.3090,  0.5968, -0.6671,  0.3980,  0.3367, -0.3479, -0.4514,  1.3409,
         -0.0590,  0.4651,  0.1409, -0.8068, -0.3012,  0.1320, -0.5763,  0.5537,
          0.2807,  0.3768,  0.6659,  0.2985, -0.9382,  0.7300, -0.1476,  0.2342,
          0.1130,  0.3445,  0.0152,  0.2738, -1.1690,  0.0831, -0.8283,  0.1238,
          0.1026, -0.2203, -0.3542, -0.5762,  0.6664,  0.5075, -0.4365, -0.1095,
         -0.2210, -1.0937,  

In [33]:
# test search result
# exact match keep the record for later step
from util.vearchutil import ImageSearch, VearchUtil
from pprint import pprint
from PIL import Image
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from util.testutil import TestUtil
import json

import torch
import torchvision.transforms as T

In [34]:
# batch test results
testutil = TestUtil(model_name="vit16")
(accuracy, cmatrix,wrong_results) = testutil.test()

pprint(cmatrix)
pprint(f"Final accuracy: {accuracy}")
print("Wrong results:")
pprint(wrong_results)


INFO 2021-11-24 18:38:56,862 testutil 260426 140455066847040 TestUtil test vit16


ValueError: not enough values to unpack (expected 4, got 3)

In [ ]:
util = VearchUtil(model_name="vgg16")
image_name="./data/zerobox/images/white/000044.png"
item = util.search_by_image(image=image_name)
pprint(item)

image = cv2.imread(image_name)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

image2 = cv2.imread(item["data"]["image_name"])
image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

testutil.plot_images([image,image2])


In [35]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests
from pprint import pprint

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

inputs = feature_extractor(images=image, return_tensors="pt")
# pprint(inputs) # dict 'pixel_values': tensor(
outputs = model(**inputs)

logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])


Predicted class: Egyptian cat


In [6]:
from transformers import ViTFeatureExtractor, ViTModel, ViTConfig
from PIL import Image
import requests
from pprint import pprint

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

# config = ViTConfig(hidden_size=512,num_attention_heads=8)
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
# model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k',config=config)

inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs,output_attentions=True,return_dict=True)
last_hidden_states = outputs.last_hidden_state

pprint(outputs.pooler_output.shape)



torch.Size([1, 768])


In [8]:
768/12

64.0